In [1]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import numpy as np
import pandas as pd
import gc
from tensorflow.keras import backend as K

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

print("TensorFlow version:", tf.__version__)

2025-12-04 14:15:10.855694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764857711.037638      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764857711.093040      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

TensorFlow version: 2.18.0


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [3]:
# Dataset paths
# For Kaggle kernel:
train = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# train = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train"
# valid = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\valid"

In [4]:
# Configuration
image_size = (256, 256)  
batch_size = 32          

print(f"Image size: {image_size}")
print(f"Batch size: {batch_size}")

Image size: (299, 299)
Batch size: 64


In [5]:
# Create datasets WITHOUT caching
def create_datasets():
    """
    Create training and validation datasets WITHOUT caching.
    Data will be loaded fresh from disk each epoch.
    Uses minimal memory but slower training.
    """
    # Training dataset
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        train,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=True
    )

    # Validation dataset
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        valid,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=False
    )

    # Save class names
    class_names = train_dataset.class_names
    num_classes = len(class_names)

    print(f"Number of classes: {num_classes}")

    total_batches = train_dataset.cardinality().numpy()

    train_dataset = train_dataset.shuffle(
        buffer_size=100,
        reshuffle_each_iteration=True
    )

    portion = 0.01
    train_dataset = train_dataset.take(int(total_batches * portion))

    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomFlip("vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomContrast(0.2),
    ])

    normalization = tf.keras.layers.Rescaling(1./255)

    train_dataset = train_dataset.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    val_dataset = val_dataset.map(
        lambda x, y: (normalization(x), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    return train_dataset, val_dataset, class_names, num_classes

print("Creating datasets...")
train_dataset, val_dataset, class_names, num_classes = create_datasets()
print("\n✓ Datasets created successfully!")

Creating datasets...
Found 70295 files belonging to 38 classes.


I0000 00:00:1764857762.504696      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 17572 files belonging to 38 classes.
Number of classes: 38

✓ Datasets created successfully!


In [6]:

def create_model(base_model_class, model_name, num_classes):

    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=image_size + (3,)
    )

    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        jit_compile=True
    )

    return model, base_model

def fine_tune_model(model, base_model, num_layers_to_unfreeze=20):
    """
    Fine-tune the model by unfreezing top layers.
    """
    base_model.trainable = True

    for layer in base_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def get_callbacks(model_name):
    """
    Create callbacks for training.
    """
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=f'{model_name}_best.weights.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]
    return callbacks

def train_single_model(model_name, base_model_class):
    """
    Train a single model with two-stage training.
    """
    print(f"\n{'='*80}")
    print(f"TRAINING {model_name}")
    print(f"{'='*80}")

    print(f"\nCreating {model_name} model...")
    model, base_model = create_model(base_model_class, model_name, num_classes)
    print(f"✓ {model_name} model created")

    print(f"\n[STAGE 1] Initial training with frozen base model...")
    initial_epochs = 10

    history_stage1 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=initial_epochs,
        callbacks=get_callbacks(f"{model_name}_stage1")
    )

    print(f"\n[STAGE 2] Fine-tuning with unfrozen top layers...")
    model = fine_tune_model(model, base_model, num_layers_to_unfreeze=20)
    fine_tune_epochs = 10

    history_stage2 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=fine_tune_epochs,
        callbacks=get_callbacks(f"{model_name}_stage2")
    )

    model.save(f"{model_name}_plant_disease_model.h5")
    print(f"\n✓ {model_name} model saved")

    print(f"\nGenerating predictions for {model_name}...")
    y_pred = model.predict(val_dataset)

    y_true = []
    for image_batch, label_batch in val_dataset:
        y_true.append(label_batch)
    y_true = tf.concat(y_true, axis=0).numpy()

    np.save(f"{model_name}_y_pred.npy", y_pred)
    np.save(f"{model_name}_y_true.npy", y_true)
    print(f"✓ {model_name} predictions saved")

    # Calculate accuracy
    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    accuracy = np.mean(y_pred_labels == y_true_labels)

    print(f"\n{model_name} FINAL VALIDATION ACCURACY: {accuracy*100:.2f}%")

    if accuracy >= 0.985:
        print(f"✓ {model_name} achieved >=98.5% accuracy!")
    else:
        print(f"✗ {model_name} did not reach 98.5% accuracy (got {accuracy*100:.2f}%)")

    # Clear memory before next model
    del model, base_model

    return accuracy

print("✓ Helper functions defined")

✓ Helper functions defined


---
## Model Training Cells
**Run each cell below to train that specific model. Skip any cell to exclude that model from training.**

---

In [ ]:
# Train VGG16
try:
    vgg16_accuracy = train_single_model('VGG16', VGG16)
except Exception as e:
    print(f"❌ Error training VGG16: {str(e)}")

In [ ]:
# Train VGG19
try:
    vgg19_accuracy = train_single_model('VGG19', VGG19)
except Exception as e:
    print(f"❌ Error training VGG19: {str(e)}")


In [ ]:
# Train InceptionV3
try:
    inceptionv3_accuracy = train_single_model('InceptionV3', InceptionV3)
except Exception as e:
    print(f"❌ Error training InceptionV3: {str(e)}")


In [ ]:
# Train Xception
try:
    xception_accuracy = train_single_model('Xception', Xception)
except Exception as e:
    print(f"❌ Error training Xception: {str(e)}")


In [7]:
# Train ResNet50
try:
    resnet50_accuracy = train_single_model('ResNet50', ResNet50)
except Exception as e:
    print(f"❌ Error training ResNet50: {str(e)}")



TRAINING ResNet50

Creating ResNet50 model...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✓ ResNet50 model created

[STAGE 1] Initial training with frozen base model...
Epoch 1/10


I0000 00:00:1764857826.969318     115 service.cc:148] XLA service 0x7d81580029d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764857826.970180     115 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1764857829.598358     115 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/10 ━━━━━━━━━━━━━━━━━━━━ 5:49 39s/step - accuracy: 0.0312 - loss: 4.2709

I0000 00:00:1764857840.148884     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.0281 - loss: 4.3379
Epoch 1: val_accuracy improved from -inf to 0.02760, saving model to ResNet50_stage1_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 93s 6s/step - accuracy: 0.0280 - loss: 4.3392 - val_accuracy: 0.0276 - val_loss: 3.7424 - learning_rate: 0.0010
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 655ms/step - accuracy: 0.0277 - loss: 4.1151
Epoch 2: val_accuracy did not improve from 0.02760
10/10 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 0.0284 - loss: 4.1147 - val_accuracy: 0.0276 - val_loss: 3.8241 - learning_rate: 0.0010
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 644ms/step - accuracy: 0.0705 - loss: 3.8106
Epoch 3: val_accuracy improved from 0.02760 to 0.02766, saving model to ResNet50_stage1_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 0.0705 - loss: 3.8094 - val_accuracy: 0.0277 - val_loss: 3.9949 - learning_rate: 0.0010
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 641ms/step - accuracy: 0.0629 - loss: 3


✓ ResNet50 model saved

Generating predictions for ResNet50...
275/275 ━━━━━━━━━━━━━━━━━━━━ 49s 162ms/step
✓ ResNet50 predictions saved

ResNet50 FINAL VALIDATION ACCURACY: 4.06%
✗ ResNet50 did not reach 98.5% accuracy (got 4.06%)


In [ ]:
# Train DenseNet121
try:
    densenet121_accuracy = train_single_model('DenseNet121', DenseNet121)
except Exception as e:
    print(f"❌ Error training DenseNet121: {str(e)}")



TRAINING DenseNet121

Creating DenseNet121 model...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✓ DenseNet121 model created

[STAGE 1] Initial training with frozen base model...
Epoch 1/10


2025-12-04 13:55:49.369125: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1253] failed to alloc 17179869184 bytes on host: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
W0000 00:00:1764856549.369186     186 device_host_allocator.h:61] could not allocate pinned host memory of size: 17179869184


---
## Results & Comparison
**Run the cells below after training your models to see the comparison.**

---

In [ ]:
# Generate comparison table
from sklearn.metrics import precision_score, recall_score, f1_score

print("="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)

# List of all possible models
all_models = ['VGG16', 'VGG19', 'InceptionV3', 'Xception', 'ResNet50', 'DenseNet121']
results = []

for model_name in all_models:
    try:
        # Load predictions
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")

        # Convert to class labels
        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)

        # Calculate metrics
        accuracy = np.mean(y_pred_labels == y_true_labels)
        precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
        recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')

        # Top-5 accuracy
        top5_pred = np.argsort(y_pred, axis=1)[:, -5:]
        top5_accuracy = np.mean([y_true_labels[i] in top5_pred[i] for i in range(len(y_true_labels))])

        results.append({
            'Model': model_name,
            'Accuracy (%)': accuracy * 100,
            'Precision (%)': precision * 100,
            'Recall (%)': recall * 100,
            'F1-Score (%)': f1 * 100,
            'Top-5 Accuracy (%)': top5_accuracy * 100,
            'Meets Target (>98.5%)': '✓' if accuracy > 0.985 else '✗'
        })
    except FileNotFoundError:
        print(f"⚠️  {model_name} was not trained (skipped)")
    except Exception as e:
        print(f"Warning: Could not load results for {model_name}: {str(e)}")

if len(results) > 0:
    # Create DataFrame
    comparison_df = pd.DataFrame(results)
    comparison_df = comparison_df.sort_values('Accuracy (%)', ascending=False)

    print("\n" + comparison_df.to_string(index=False))
    print("\n" + "="*80)

    # Count models above target
    models_above_target = comparison_df[comparison_df['Meets Target (>98.5%)'] == '✓'].shape[0]
    print(f"\nModels achieving >98.5% accuracy: {models_above_target}/{len(results)}")

    # Save comparison table
    comparison_df.to_csv('model_comparison_sequential.csv', index=False)
    print("\n✓ Comparison table saved to 'model_comparison_sequential.csv'")
else:
    print("\n⚠️  No models were trained. Please run at least one model training cell above.")

In [ ]:
# Visualize comparison
import matplotlib.pyplot as plt
import seaborn as sns

if len(results) > 0:
    # Set style
    sns.set_style('whitegrid')
    plt.figure(figsize=(12, 6))

    # Plot accuracy comparison
    plt.subplot(1, 2, 1)
    colors = ['green' if x == '✓' else 'red' for x in comparison_df['Meets Target (>98.5%)']]
    plt.barh(comparison_df['Model'], comparison_df['Accuracy (%)'], color=colors, alpha=0.7)
    plt.axvline(x=98.5, color='blue', linestyle='--', label='Target (98.5%)')
    plt.xlabel('Accuracy (%)')
    plt.title('Model Accuracy Comparison')
    plt.legend()
    plt.tight_layout()

    # Plot metrics comparison
    plt.subplot(1, 2, 2)
    metrics_df = comparison_df[['Model', 'Precision (%)', 'Recall (%)', 'F1-Score (%)']].set_index('Model')
    metrics_df.plot(kind='bar', ax=plt.gca(), width=0.8)
    plt.ylabel('Score (%)')
    plt.title('Model Metrics Comparison')
    plt.xticks(rotation=45, ha='right')
    plt.legend(loc='lower right')
    plt.tight_layout()

    plt.savefig('model_comparison_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ Visualization saved to 'model_comparison_visualization.png'")
else:
    print("⚠️  No results to visualize. Train at least one model first.")

In [ ]:
# Generate detailed classification reports
from sklearn.metrics import classification_report

print("="*80)
print("DETAILED CLASSIFICATION REPORTS")
print("="*80)

for model_name in all_models:
    try:
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")

        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)

        print(f"\n{'='*80}")
        print(f"{model_name} - Classification Report")
        print(f"{'='*80}")

        report = classification_report(y_true_labels, y_pred_labels, target_names=class_names)
        print(report)

        # Save report
        with open(f"{model_name}_classification_report.txt", 'w') as f:
            f.write(f"{model_name} - Classification Report\n")
            f.write("="*80 + "\n")
            f.write(report)

        print(f"✓ Report saved to '{model_name}_classification_report.txt'")
    except FileNotFoundError:
        print(f"\n⚠️  {model_name} was not trained (skipped)")
    except Exception as e:
        print(f"Warning: Could not generate report for {model_name}: {str(e)}")

In [ ]:
# Final summary
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

if len(results) > 0:
    print(f"\nTotal models trained: {len(results)}")
    print(f"Models achieving >98.5% accuracy: {models_above_target}")
    print(f"\nBest performing model: {comparison_df.iloc[0]['Model']}")
    print(f"Best accuracy: {comparison_df.iloc[0]['Accuracy (%)']:.2f}%")
else:
    print("\n⚠️  No models were trained.")

print("\nKey features implemented:")
print("  ✓ Sequential training (one model at a time)")
print("  ✓ Separate cells for each model (skip any model)")
print("  ✓ NO caching (minimal memory usage)")
print("  ✓ Memory management between models")
print("  ✓ GPU memory growth enabled")
print("  ✓ Mixed precision training")
print("  ✓ Two-stage training with fine-tuning")
print("  ✓ Enhanced data augmentation")
print("  ✓ Learning rate scheduling")
print("  ✓ Early stopping and model checkpointing")
print("\n" + "="*80)
print("TRAINING COMPLETE!")
print("="*80)